# Zero Shot Text Classification

Large language model a lot of things already and you really need to finetune them to do the task that you want. This is an example of zero-shot text classification for something like deciding what the sentiment of a response to a survey is.

In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import torch
import torch.nn.functional as f
from sentence_transformers import SentenceTransformer


text_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
orig_target_classes = ['happy', 'sad', 'angry', 'disgust', 'fear']
target_classes = [f'the text describes {emotion}' for emotion in orig_target_classes]
target_vectors = text_model.encode(target_classes, convert_to_tensor=True)
print(target_vectors.shape)

torch.Size([5, 384])


In [4]:
sentences = [
    'I like blue books',
    'I was kinda in a depression eating mood so I went to the grocery to buy some junk food.',
    'I can\'t believe that person just threw their trash on the floor and walked away without cleaning it up!'
]
sentences_vectors = text_model.encode(sentences, convert_to_tensor=True)
print(sentences_vectors.shape)

torch.Size([3, 384])


## Perform the zero-shot classification

In [5]:
zeroshot_logits = sentences_vectors @ target_vectors.T
print(zeroshot_logits)

pred_prob = f.softmax(zeroshot_logits, dim=1)
print(pred_prob)

for text, pi in zip(sentences, pred_prob.argmax(dim=1)):
    print(f'Text: {text}')
    print(f'Emotion: {orig_target_classes[pi]}')
    print()

tensor([[0.1564, 0.0687, 0.1056, 0.0922, 0.1485],
        [0.2238, 0.2781, 0.1367, 0.1762, 0.0183],
        [0.0173, 0.0964, 0.0944, 0.2078, 0.0612]])
tensor([[0.2085, 0.1910, 0.1982, 0.1955, 0.2068],
        [0.2110, 0.2227, 0.1934, 0.2012, 0.1718],
        [0.1846, 0.1998, 0.1994, 0.2233, 0.1929]])
Text: I like blue books
Emotion: happy

Text: I was kinda in a depression eating mood so I went to the grocery to buy some junk food.
Emotion: sad

Text: I can't believe that person just threw their trash on the floor and walked away without cleaning it up!
Emotion: disgust

